In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
from main_io_comp_vis_all import calibrate_magnetic_wifi_ibeacon_to_position, read_data_file
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sample_sub = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv')
sample_sub[["site","path","time"]]=sample_sub["site_path_timestamp"].str.split('_', expand=True)
site_list = list(set(list(sample_sub["site"])))

# 1) Creating Train Data

## 1.1) Initialize

In [ ]:
floor_txt_num = {'F1':1,'1F':1,'L1':1,'1L':1,'F2':3,'2F':3,'L2':3,'2L':3,'F3':4,'3F':4,'L3':4,'3L':4,'F4':5,'4F':5,'L4':5,'4L':5,
                'F5':6,'5F':6,'L5':6,'5L':6,'F6':7,'6F':7,'L6':7,'6L':7,'F7':8,'7F':8,'L7':8,'7L':8,'F8':9,'8F':9,'L8':9,'8L':9,
                'F9':10,'9F':10,'L9':10,'9L':10,'F10':11,'10F':11,'L10':11,'10L':11,'F11':12,'11F':12,'L11':12,'11L':12,
                'B1':1,'1B':1,"B":1,'B2':2,'2B':2,'B3':3,'3B':3,'B4':4,'4B':4}

In [ ]:
train_df = pd.DataFrame(data={"x":[],"y":[],"floor":[],"site":[]})
NUM_FEATS=10
BSSID_FEATS = [f'BSSID_{i}' for i in range(NUM_FEATS)]
RSSI_FEATS = [f'RSSI_{i}' for i in range(NUM_FEATS)]
BEACON_FEATS = [f'BEACONID_{i}' for i in range(NUM_FEATS)]
BEACONRSSI_FEATS = [f'BEACONRSSI_{i}' for i in range(NUM_FEATS)]
train_df[BSSID_FEATS+RSSI_FEATS+BEACON_FEATS+BEACONRSSI_FEATS]=[]
train_df[['site']+BSSID_FEATS+BEACON_FEATS]=train_df[['site']+BSSID_FEATS+BEACON_FEATS].astype('str')
train_df_row=train_df.copy()

## 1.2) Create DFrame

In [ ]:
for idxs, site in enumerate(site_list):
    floor_list = os.listdir('../input/indoor-location-navigation/train/'+site)
    for idxf, floor in enumerate(floor_list):
        train_paths = glob.glob('../input/indoor-location-navigation/train/'+site+'/'+floor+'/*')
        print("Site:",idxs," /24 ; Floor:",idxf)
        for path in train_paths:
            data = read_data_file(path)
            wp_df = pd.DataFrame(data=data.waypoint, columns=["Time","x","y"])
            if (data.wifi != []):
                wifi_df = pd.DataFrame(data=data.wifi, columns= ["Time","SSID","BSSID","RSSI","LTime"])
            else:
                wifi_df = pd.DataFrame(data={"Time":[],"SSID":[],"BSSID":[],"RSSI":[],"LTime":[]})
            if (data.ibeacon != []):
                beacon_df = pd.DataFrame(data=data.ibeacon, columns= ["Time","BEACONID","RSSI"])
            else:
                beacon_df = pd.DataFrame(data={"Time":[],"BEACONID":[],"RSSI":[]})
            
            for idx, row in wp_df.iterrows():
                wifi_df_wp = wifi_df.loc[(wifi_df["Time"].astype('int')<int(row["Time"]+2000)) & (wifi_df["Time"].astype('int')>int(row["Time"]-2000)),:]
                wifi_df_wp[["Time","RSSI"]] = wifi_df_wp[["Time","RSSI"]].astype('int')
                wifi_df_wp = wifi_df_wp.drop(columns=["SSID","LTime"])
                wifi_df_wp=wifi_df_wp.groupby(by=["BSSID"]).mean()
                wifi_df_wp = wifi_df_wp.sort_values(by=["RSSI"],ascending=False)
                wifi_df_wp = wifi_df_wp[:10]
                train_df_row.loc[0,['x','y']]=row[['x','y']]
                train_df_row.loc[0,['floor']]= floor_txt_num[floor]
                train_df_row.loc[0,['site']]= site
                train_df_row_bssid = list(wifi_df_wp.index)
                train_df_row_rssi = list(wifi_df_wp["RSSI"])
                for i in range(10-len(train_df_row_bssid)):
                    train_df_row_bssid.append('XXX')
                    train_df_row_rssi.append(-100)    
                train_df_row.loc[0,BSSID_FEATS]=train_df_row_bssid
                train_df_row.loc[0,RSSI_FEATS]=train_df_row_rssi
                #print(train_df_row_bssid)

                beacon_df_wp = beacon_df.loc[(beacon_df["Time"].astype('int')<int(row["Time"]+2000)) & (beacon_df["Time"].astype('int')>int(row["Time"]-2000)),:]
                beacon_df_wp[["Time","RSSI"]] = beacon_df_wp[["Time","RSSI"]].astype('int')
                beacon_df_wp=beacon_df_wp.groupby(by=["BEACONID"]).mean()
                beacon_df_wp = beacon_df_wp.sort_values(by=["RSSI"],ascending=False)
                beacon_df_wp = beacon_df_wp[:10]
                train_df_row_beaconid = list(beacon_df_wp.index)
                train_df_row_brssi = list(beacon_df_wp["RSSI"])
                for i in range(10-len(train_df_row_beaconid)):
                    train_df_row_beaconid.append('XXX')
                    train_df_row_brssi.append(-100)    
                train_df_row.loc[0,BEACON_FEATS]=train_df_row_beaconid
                train_df_row.loc[0,BEACONRSSI_FEATS]=train_df_row_brssi
                # print(train_df_row_beaconid)
                # print(train_df_row)
                train_df = train_df.append(train_df_row, ignore_index=True)
train_df.to_csv("Train_Data.csv")            